# Gaussian processes
Na podstawie https://peterroelants.github.io/posts/gaussian-process-kernel-fitting/

## Importy

In [1]:
import os
from itertools import islice
import warnings
warnings.simplefilter("ignore")

import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_probability as tfp

from tqdm.notebook import tqdm
import bokeh
import bokeh.io
import bokeh.plotting
import bokeh.models
from IPython.display import display, HTML

bokeh.io.output_notebook(hide_banner=True)

tfb = tfp.bijectors
tfd = tfp.distributions
tfk = tfp.math.psd_kernels

np.random.seed(42)
tf.random.set_seed(42)

## Załadowanie danych

In [2]:
df = pd.read_csv("monthly_in_situ_co2_mlo.csv", skiprows=200)

co2_df = df[df.columns[3:5]]
co2_df = co2_df.rename(columns={co2_df.columns[0]: 'Date', co2_df.columns[1]: 'CO2'})

co2_df

,Date,CO2
0,1970.0411,325.06
1,1970.1260,325.98
2,1970.2027,326.93
3,1970.2877,328.13
4,1970.3699,328.08
...,...,...
631,2022.6219,-99.99
632,2022.7068,-99.99
633,2022.7890,-99.99
634,2022.8740,-99.99


## Podział na zbiór zaobserwowany i predyktowany

In [3]:
fig = bokeh.plotting.figure(
    width=900, height=600, 
    x_range=(1970, 2021), y_range=(300, 420))
fig.xaxis.axis_label = 'Rok'
fig.yaxis.axis_label = 'CO₂'
fig.add_layout(bokeh.models.Title(
    text='Atmosferyczna koncetracja CO₂', 
    text_font_size="14pt"), 'above')
fig.line(
    co2_df.Date, co2_df.CO2, legend_label='Dane',
    line_width=2, line_color='midnightblue')
fig.legend.location = 'top_left'
fig.toolbar.autohide = True
bokeh.plotting.show(fig)

In [4]:
date_split_predict = 2010
df_observed = co2_df[co2_df.Date < date_split_predict]
print('{} measurements in the observed set'.format(len(df_observed)))
df_predict = co2_df[co2_df.Date >= date_split_predict]
print('{} measurements in the test set'.format(len(df_predict)))

480 measurements in the observed set
156 measurements in the test set


## Średnia

In [5]:
observations_mean = tf.constant(
    [np.mean(df_observed.CO2.values)], dtype=tf.float64)
mean_fn = lambda _: observations_mean

2022-01-29 19:38:53.058006: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Kernel

In [6]:
constrain_positive = tfb.Shift(np.finfo(np.float64).tiny)(tfb.Exp())

# Exponential Quadratic kernel
smooth_amplitude = tfp.util.TransformedVariable(
    initial_value=10., bijector=constrain_positive, dtype=np.float64,
    name='smooth_amplitude')
smooth_length_scale = tfp.util.TransformedVariable(
    initial_value=10., bijector=constrain_positive, dtype=np.float64,
    name='smooth_length_scale')
smooth_kernel = tfk.ExponentiatedQuadratic(
    amplitude=smooth_amplitude,
    length_scale=smooth_length_scale)

# LocalPeriodickernel
periodic_amplitude = tfp.util.TransformedVariable(
    initial_value=5.0, bijector=constrain_positive, dtype=np.float64,
    name='periodic_amplitude')
periodic_length_scale = tfp.util.TransformedVariable(
    initial_value=1.0, bijector=constrain_positive, dtype=np.float64,
    name='periodic_length_scale')
periodic_period = tfp.util.TransformedVariable(
    initial_value=1.0, bijector=constrain_positive, dtype=np.float64,
    name='periodic_period')
periodic_local_length_scale = tfp.util.TransformedVariable(
    initial_value=1.0, bijector=constrain_positive, dtype=np.float64,
    name='periodic_local_length_scale')
local_periodic_kernel = (
    tfk.ExpSinSquared(
        amplitude=periodic_amplitude,
        length_scale=periodic_length_scale,
        period=periodic_period) *
    tfk.ExponentiatedQuadratic(
        length_scale=periodic_local_length_scale))

# RationalQuadratic kernel
irregular_amplitude = tfp.util.TransformedVariable(
    initial_value=1., bijector=constrain_positive, dtype=np.float64,
    name='irregular_amplitude')
irregular_length_scale = tfp.util.TransformedVariable(
    initial_value=1., bijector=constrain_positive, dtype=np.float64,
    name='irregular_length_scale')
irregular_scale_mixture = tfp.util.TransformedVariable(
    initial_value=1., bijector=constrain_positive, dtype=np.float64,
    name='irregular_scale_mixture')
irregular_kernel = tfk.RationalQuadratic(
    amplitude=irregular_amplitude,
    length_scale=irregular_length_scale,
    scale_mixture_rate=irregular_scale_mixture)

# White Noise kernel
observation_noise_variance = tfp.util.TransformedVariable(
    initial_value=1, bijector=constrain_positive, dtype=np.float64,
    name='observation_noise_variance')

trainable_variables = [v.variables[0] for v in [
    smooth_amplitude,
    smooth_length_scale,
    periodic_amplitude,
    periodic_length_scale,
    periodic_period,
    periodic_local_length_scale,
    irregular_amplitude,
    irregular_length_scale,
    irregular_scale_mixture,
    observation_noise_variance
]]
kernel = (smooth_kernel + local_periodic_kernel + irregular_kernel)

2022-01-29 19:38:54.854769: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


## Hiperperametry

In [7]:
batch_size = 128

batched_dataset = (
    tf.data.Dataset.from_tensor_slices(
        (df_observed.Date.values.reshape(-1, 1), df_observed.CO2.values))
    .shuffle(buffer_size=len(df_observed))
    .repeat(count=None)
    .batch(batch_size)
)

In [8]:
@tf.function(autograph=False, experimental_compile=False)
def gp_loss_fn(index_points, observations):
    """Gaussian process negative-log-likelihood loss function."""
    gp = tfd.GaussianProcess(
        mean_fn=mean_fn,
        kernel=kernel,
        index_points=index_points,
        observation_noise_variance=observation_noise_variance
    )

    negative_log_likelihood = -gp.log_prob(observations)
    return negative_log_likelihood

In [9]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

batch_nlls = []
full_ll = []
nb_iterations = 10000
for i, (index_points_batch, observations_batch) in tqdm(
        enumerate(islice(batched_dataset, nb_iterations)), total=nb_iterations):
    with tf.GradientTape() as tape:
        loss = gp_loss_fn(index_points_batch, observations_batch)
    grads = tape.gradient(loss, trainable_variables)
    optimizer.apply_gradients(zip(grads, trainable_variables))
    batch_nlls.append((i, loss.numpy()))
    if i % 100 == 0:
        ll = gp_loss_fn(
            index_points=df_observed.Date.values.reshape(-1, 1),
            observations=df_observed.CO2.values)
        full_ll.append((i, ll.numpy()))

  0%|          | 0/10000 [00:00<?, ?it/s]

Instructions for updating:
`jitter` is deprecated; please use `marginal_fn` directly.


In [10]:
fig = bokeh.plotting.figure(
    width=900, height=500,
    x_range=(0, nb_iterations), y_range=(60, 250))
fig.add_layout(bokeh.models.Title(
    text='Ujemny Log-Likehood (NLL) podczas treningu',
    text_font_size="14pt"), 'above')
fig.xaxis.axis_label = 'Iteracja'
fig.yaxis.axis_label = 'NLL batch'
fig.line(
    *zip(*batch_nlls), legend_label='Batch dane',
    line_width=2, line_color='midnightblue')
fig.extra_y_ranges = {
    'fig1ax2': bokeh.models.Range1d(start=50, end=250)}
fig.line(
    *zip(*full_ll), legend_label='Wszystkie zaobserwowane dane',
    line_width=2, line_color='red', y_range_name='fig1ax2')
fig.add_layout(bokeh.models.LinearAxis(
    y_range_name='fig1ax2', axis_label='NLL all'), 'right')

fig.legend.location = 'top_right'
fig.toolbar.autohide = True
bokeh.plotting.show(fig)

In [11]:
variables = [
    smooth_amplitude,
    smooth_length_scale,
    periodic_amplitude,
    periodic_length_scale,
    periodic_period,
    periodic_local_length_scale,
    irregular_amplitude,
    irregular_length_scale,
    irregular_scale_mixture,
    observation_noise_variance
]

data = list([(var.variables[0].name[:-2], var.numpy()) for var in variables])
df_variables = pd.DataFrame(
    data, columns=['Hiperparametr', 'Wartość'])
display(HTML(df_variables.to_html(
    index=False, float_format=lambda x: f'{x:.4f}')))

Hiperparametr,Wartość
smooth_amplitude,115.33117344835402
smooth_length_scale,92.76501596654424
periodic_amplitude,2.7805954949602736
periodic_length_scale,1.545718248369088
periodic_period,0.9998201802925835
periodic_local_length_scale,64.58712075395604
irregular_amplitude,0.9998068033738385
irregular_length_scale,1.2706274880338722
irregular_scale_mixture,0.1324588504639927
observation_noise_variance,0.04972014556325733


## Predykcje posterior

In [12]:
gp_posterior_predict = tfd.GaussianProcessRegressionModel(
    mean_fn=mean_fn,
    kernel=kernel,
    index_points=df_predict.Date.values.reshape(-1, 1),
    observation_index_points=df_observed.Date.values.reshape(-1, 1),
    observations=df_observed.CO2.values,
    observation_noise_variance=observation_noise_variance)

# Posterior mean and standard deviation
posterior_mean_predict = gp_posterior_predict.mean()
posterior_std_predict = gp_posterior_predict.stddev()

In [13]:
μ = posterior_mean_predict.numpy()
σ = posterior_std_predict.numpy()

# Plot
fig = bokeh.plotting.figure(
    width=600, height=400,
    x_range=(2012, 2021.3), y_range=(384, 418))
fig.xaxis.axis_label = 'Rok'
fig.yaxis.axis_label = 'CO₂ (ppm)'
fig.add_layout(bokeh.models.Title(
    text='Prognozy posterior uwarunkowane obserwacjami po 2012 r.',
    text_font_style="italic"), 'above')
fig.add_layout(bokeh.models.Title(
    text='Atmosferyczna koncetracja CO₂',
    text_font_size="14pt"), 'above')
fig.circle(
    co2_df.Date, co2_df.CO2, legend_label='Prawdziwe dane',
    size=2, line_color='midnightblue')
fig.line(
    df_predict.Date.values, μ, legend_label='μ (prognozy)',
    line_width=2, line_color='firebrick')
# Prediction interval
band_x = np.append(
    df_predict.Date.values, df_predict.Date.values[::-1])
band_y = np.append(
    (μ + 2*σ), (μ - 2*σ)[::-1])
fig.patch(
    band_x, band_y, color='firebrick', alpha=0.4,
    line_color='firebrick', legend_label='2σ')

fig.legend.location = 'top_left'
fig.toolbar.autohide = True
bokeh.plotting.show(fig)

## Komentarz wyników

Widać, że utworzony model poprawnie nauczył się oraz z wysoką skutecznością oszacowywał koncentrację CO2 w danych latach. Nie wszystkie wyniki udało się zmieścić w przedziale niepewności w późniejszych latach. Podejrzewam, że może wynikać to z faktu, że ilość CO2 przewyższa oczekiwania w tych latach - nie bez powodu jest to problem obecnego świata. 

# Notowania giełdowe

In [14]:
# Z https://stackoverflow.com/questions/24588437/convert-date-to-float-for-linear-regression-on-pandas-data-frame
def dt64_to_float(dt64):
    """Converts numpy.datetime64 to year as float.

    Rounded to days

    Parameters
    ----------
    dt64 : np.datetime64 or np.ndarray(dtype='datetime64[X]')
        date data

    Returns
    -------
    float or np.ndarray(dtype=float)
        Year in floating point representation
    """

    year = dt64.astype('M8[Y]')
    # print('year:', year)
    days = (dt64 - year).astype('timedelta64[D]')
    # print('days:', days)
    year_next = year + np.timedelta64(1, 'Y')
    # print('year_next:', year_next)
    days_of_year = (year_next.astype('M8[D]') - year.astype('M8[D]')
                    ).astype('timedelta64[D]')
    # print('days_of_year:', days_of_year)
    dt_float = 1970 + year.astype(float) + days / (days_of_year)
    # print('dt_float:', dt_float)
    return dt_float

In [15]:
df_b = pd.read_csv("wig20_d.csv")
df_b = df_b[df_b.columns[0:2]]
df_b = df_b.rename(columns={df_b.columns[0]: 'Date', df_b.columns[1]: 'Value'})
df_b['Date'] = pd.to_datetime(df_b['Date'])
df_b['Date'] = dt64_to_float(df_b['Date'].to_numpy())
df_b['Value'] = df_b['Value'].astype(float)

df_b

,Date,Value
0,1991.287671,100.00
1,1991.306849,95.70
2,1991.326027,93.50
3,1991.364384,92.90
4,1991.383562,95.50
...,...,...
6963,2020.923497,1894.40
6964,2020.931694,1949.61
6965,2020.934426,1941.14
6966,2020.937158,1976.65


In [16]:
fig = bokeh.plotting.figure(
    width=900, height=600, 
    x_range=(1991, 2021), y_range=(0, 4000))
fig.xaxis.axis_label = 'Rok'
fig.yaxis.axis_label = 'Wartość'
fig.add_layout(bokeh.models.Title(
    text='Notowania giełdowe WIG20 ', 
    text_font_size="14pt"), 'above')
fig.line(
    df_b.Date, df_b.Value, legend_label='Dane',
    line_width=2, line_color='midnightblue')
fig.legend.location = 'top_left'
fig.toolbar.autohide = True
bokeh.plotting.show(fig)

In [17]:
date_split_predict = 2019
df_observed = df_b[df_b.Date < date_split_predict]
print('{} measurements in the observed set'.format(len(df_observed)))
df_predict = df_b[df_b.Date >= date_split_predict]
print('{} measurements in the test set'.format(len(df_predict)))

observations_mean = tf.constant(
    [np.mean(df_observed.Value.values)], dtype=tf.float64)
mean_fn = lambda _: observations_mean

6480 measurements in the observed set
488 measurements in the test set


In [18]:
batch_size = 128

batched_dataset = (
    tf.data.Dataset.from_tensor_slices(
        (df_observed.Date.values.reshape(-1, 1), df_observed.Value.values))
    .shuffle(buffer_size=len(df_observed))
    .repeat(count=None)
    .batch(batch_size)
)

### Niestety nie zdążyłem zrobić zrobić eksperymentu

In [19]:
df_c = pd.read_csv("cmr_d.csv")
df_c = df_c[df_c.columns[0:2]]
df_c = df_c.rename(columns={df_c.columns[0]: 'Date', df_c.columns[1]: 'Value'})
df_c['Date'] = pd.to_datetime(df_c['Date'])
df_c['Date'] = dt64_to_float(df_c['Date'].to_numpy())
df_c['Value'] = df_c['Value'].astype(float)

df_c

,Date,Value
0,1999.186301,21.756
1,1999.189041,23.766
2,1999.191781,26.053
3,1999.200000,28.612
4,1999.202740,28.612
...,...,...
5440,2020.931694,198.000
5441,2020.934426,196.000
5442,2020.937158,196.000
5443,2020.939891,196.500


In [20]:
fig = bokeh.plotting.figure(
    width=900, height=600, 
    x_range=(1999, 2021), y_range=(0, 250))
fig.xaxis.axis_label = 'Rok'
fig.yaxis.axis_label = 'Wartość'
fig.add_layout(bokeh.models.Title(
    text='Notowania giełdowe COMARCH', 
    text_font_size="14pt"), 'above')
fig.line(
    df_c.Date, df_c.Value, legend_label='Dane',
    line_width=2, line_color='midnightblue')
fig.legend.location = 'top_left'
fig.toolbar.autohide = True
bokeh.plotting.show(fig)

In [21]:
date_split_predict = 2020
df_observed = df_c[df_c.Date < date_split_predict]
print('{} measurements in the observed set'.format(len(df_observed)))
df_predict = df_c[df_c.Date >= date_split_predict]
print('{} measurements in the test set'.format(len(df_predict)))

5204 measurements in the observed set
241 measurements in the test set


### Niestety nie zdążyłem zrobić zrobić eksperymentu